In [16]:
import json
import requests

weat = requests.get('http://adapt2.sis.pitt.edu/pcex-authoring/api/hub').json()

In [17]:
bs_users = ["ahromic1@etf.unsa.ba","mkamberovi1@etf.unsa.ba"]
en_users = ["adityajoshi5@gmail.com"]
weat = [item for item in weat if item['author']['email'] in [*bs_users, *en_users]]

In [22]:
import re

pl = {
    "python": "Python",
    "java": "Java",
    "cpp": "Cpp",
    "c": "C",
}

def clean(name):
    name = re.sub(r'__([a-f0-9]{24})-([a-f0-9]{24})-([0-9]{1,2})$', '', name)
    return re.sub(r'__([a-f0-9]{24})$', '', name).strip()

def get_acos_url(item, activity):
    item_nameid = activity['activityName'].replace(' ', '_').replace('.', '_').replace('(', '').replace(')', '')
    if item['id'] not in item_nameid:
        item_nameid += f"__{item['id']}"
    return f"https://acos.cs.vt.edu/html/acos-pcex/acos-pcex-examples/{item_nameid}"

urls = []

# api_token = open('api_token.txt').read().strip()
for item in weat:
    activity = requests.get('http://adapt2.sis.pitt.edu/pcex-authoring/api/hub/'+item['id']).json()
    if type(activity) != list or len(activity) == 0:
        continue
    activity = activity[0]
    
    pending = []
    pending.append({
        "identity": {
            "id": activity['activityName'],
            "title": clean(activity['activityName']),
            "type": "PCEX Set"
        },
        "tags": [],
        "status": "public",
        "user_email": "moh70@pitt.edu",
        "links": {"demo_url": get_acos_url(item, activity)},
        "attribution": {
            "created_at": "2026-01-01T00:00:00Z",
            "provider": "PCEX",
            "publisher": "PAWS Lab - WEAT",
            "authors": [{"name": item['author']['fullname'],"affiliation": ""}]
        },
        "languages": {
            "content_language": "bs" if bs_users else "en",
            "programming_languages": [pl[activity['language'].lower()]]
        },
        "content": {"prompt": "","source_code": ""},
        "classification": {"topics": [],"difficulty": "","knowledge_components": {}},
        "pedagogy": {"learning_objectives": [],"instructional_role": "","prerequisites": {"topics": [],"concepts": [],"item_ids": []}},
        "interaction": {"interaction_type": ""},
        "delivery": [{"format": "html","url": get_acos_url(item, activity)}],
        "rights": {"license": "MIT","license_url": "","usage_notes": ""},
        "uses": [],
    })
    
    for index, goal in enumerate(activity['activityGoals']):
        pending.append({
            "identity": {
                "id": goal['name'],
                "title": clean(goal['name']),
                "type": "PCEX Example" if goal['fullyWorkedOut'] else "PCEX Challenge"
            },
            "tags": [],
            "status": "public",
            "user_email": "moh70@pitt.edu",
            "links": {"demo_url": get_acos_url(item, activity)+f"?index={index}"},
            "attribution": {
                "created_at": "2026-01-01T00:00:00Z",
                "provider": "PCEX",
                "publisher": "PAWS Lab - WEAT",
                "authors": [{"name": item['author']['fullname'],"affiliation": ""}]
            },
            "languages": {
                "content_language": "bs" if bs_users else "en",
                "programming_languages": [pl[activity['language'].lower()]]
            },
            "content": {
                "prompt": goal['goalDescription'],
                "source_code": '\n'.join([
                    line['content']
                    for line in sorted(goal['lineList'], key=lambda x: x['number'])
                ])
            },
            "classification": {"topics": [],"difficulty": "","knowledge_components": {}},
            "pedagogy": {"learning_objectives": [],"instructional_role": "","prerequisites": {"topics": [],"concepts": [],"item_ids": []}},
            "interaction": {"interaction_type": ""},
            "delivery": [{"format": "html","url": get_acos_url(item, activity)+f"?index={index}"}],
            "rights": {"license": "MIT","license_url": "","usage_notes": ""},
            "uses": [],
        })
        
    for each in pending:
        urls.append(each['links']['demo_url'])
    #     response = requests.post(
    #         f"http://adapt2.sis.pitt.edu/next.course-authoring/api/slc-items-api/",
    #         json=each,
    #         headers={'api-token': api_token}
    #     )
    #     print('status_code:', response.status_code, ' --> ', each['identity']['title'])